Logistic Regression Cow Classifier

In [ ]:
#Image classifer with logistic regression
#images shape(width,height,3(rgb))

import numpy as np
import scipy 
import os
import matplotlib.pyplot as plot
import math
from scipy import ndimage

# TO-DO
# fetch a ton more images, of your own choice / category 
# create a train and a test set <---- write code for this 

In [ ]:
# Global Constants
side_length = 650
image_path = "./untitled folder/"
test_ratio = .2

In [ ]:
#
test0 = "./untitled folder/" + "cow0.jpg" #standardize file extensions
test1 = "./untitled folder/" + "notCow0.jpg" 
test0Array = scipy.ndimage.imread(test0,flatten=False)
test1Array = scipy.ndimage.imread(test1,flatten=False) 
print("Shape of Test Image 0:", np.shape(test0Array))
print("Shape of Test Image 1:", np.shape(test1Array))

# Resize input image arrays 
test0Array_resized = scipy.misc.imresize(test0Array,size=(side_length,side_length,3))
test1Array_resized = scipy.misc.imresize(test1Array,size=(side_length,side_length,3))

In [ ]:
# Read in input image files into arrays, validate inputs
images = os.listdir(image_path)

# First, build dataset for training data
examples = []
labels = []
for image in images:
    if image.split(".")[-1] != "DS_Store":
        cur_label = 0
        if image[0] == "c":
            cur_label = 1
        labels += [cur_label]
        cur_image_path = image_path + image
        cur_array = scipy.ndimage.imread(cur_image_path,flatten=False) #reads image as numpy array (lenght,height,3)
        cur_array_resized = scipy.misc.imresize(cur_array,size=(side_length,side_length,3)) #resizes images to a uniform shape
        examples += [cur_array_resized] 
# print(labels)

print("Shape of train_examples:",np.shape(examples)) #should be (len(images),side_length,side_length,3)

# Then, build dataset for train and test data...
number_test_examples = int(len(examples)*test_ratio)
test_examples = examples[:number_test_examples]
train_examples = examples[number_test_examples:]

train_examples = np.array(train_examples) #converts the list of examples to an array
test_examples = np.array(test_examples) 


# Next, we flatten our train and test examples
# .T => transposes our inputs
# (a, b, c, d) ----> (b * c * d, a)

flattened_train_examples = train_examples.reshape(train_examples.shape[0], -1).T
flattened_test_examples = test_examples.reshape(test_examples.shape[0], -1).T
# print(type(train_examples[0]))
# flattened_train_examples = train_examples.reshape(train_examples[0],-1).T
#fattened_train_examples = train_examples.flatten()
print(np.shape(flattened_train_examples))
print(np.shape(flattened_test_examples))

# print("Shape of test0Array_resized:",np.shape(test0Array_resized))

# Standardize color values of the image 
standardized_train_examples = flattened_train_examples/255 #225 is the maximum rgb value/ This is done to decrease varaince in inputs thus more efficint
standardized_test_examples = flattened_test_examples/255
print(standardized_train_examples)


# Create labels for data
# labels = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0]
labels_train  = np.array(labels[:21])
labels_test = np.array(labels[21:])
# Randomize and shuffle labels/data

In [127]:
def sigmoid(x):
    print("sigmoid input",x)
    output = 1.0/(1.0+np.exp(-x))
    print("sigmoid output",output)
    return output
    
sigmoid(1)
# write test cases for this function so that it works for np arrays 

sigmoid input 1
sigmoid output 0.73105857863


0.7310585786300049

In [ ]:
def initialize_zeros(dimension): #dimension = number of examples
    zeroed_weights = np.zeros((dimension, 1))
    zeroed_baisies = 0. #baisies always start at 0 b/c they are developed as the model trains
    
    # asserts here, use isinstance()
    # check shape of zeroed weight
    assert(isinstance(zeroed_baisies, float) or isinstance(zeroed_baisies, int))
    
    return zeroed_weights, zeroed_baisies
    
initialize_zeros(10)
# write test cases for this function <--- use asserts, check that zeored_weights and zeored_baisies are of the right
# type. 

In [129]:
def propagate(X,Y,w,b):
    # b = scalar
    # w = (____, 1)
    # X = (____, examples)
    # Y = (1, examples)
#     print("propagate shapes")
#     print("X",np.shape(X))
#     print("Y",np.shape(Y))
#     print("w",np.shape(w))
#     print("b",np.shape(b))
    
    w_transposed = w.T
    num_of_examples = np.shape(X)[1]
    
    dot_product = np.dot(w_transposed,X) #dot product of X and w ORDER MATTERS
#     print("Dot product",dot_product)
    z = dot_product + b
#     print("z",z)
    a = sigmoid(z) #in begining should be 0
#     print("a",a)
    print("propgate: a",a)
    print((-1/num_of_examples)*np.sum((1-Y)*np.log(1-a),"HERERERRER"))
    loss = (-1/num_of_examples)*np.sum((1-Y)*np.log(1-a) + (Y *np.log(a))) #log likelyhood cost for logistic regression
    print("loss",loss)
    dw = (1/num_of_examples) * np.dot(X,(a-Y).T)
    #
    db = (1/num_of_examples) * np.sum(a-Y) #np.sum collapes input
    #
    loss = np.squeeze(loss)
    #
    grads = {"dw":dw,"db":db}
    return grads,loss    


In [ ]:
w,b = initialize_zeros(np.shape(standardized_train_examples)[0])
print(np.shape(standardized_train_examples),"in good call stardard train examples")
propagate(standardized_train_examples,labels_train,w,b)


In [ ]:
def optimizer(w, b, X, Y, learning_rate, num_iterations, print_results=True):
    # b = scalar
    # w = (____, 1)
    # X = (____, examples)
    # Y = (1, examples)
    costs = []
#     print("X in optimizer",np.shape(X),"Y in optimizer",np.shape(Y))
    for i in range(num_iterations):
        grads, cost = propagate(X,Y,w,b)
        print(cost)
        dw = grads["dw"]
        db = grads["db"]
        w = w - (learning_rate * dw)
        b = b - (learning_rate * db)
        if i % 50 == 0:
            costs += [cost]
            print("cost #",i,":", costs)
    print("Finsihed :)" , costs )
    grads = {"dw":dw,"db":db}
    params = {"w":w,"b":b}
    return grads,params,costs

In [ ]:
print("shapes before optimizer")
print("X Shape", np.shape(standardized_train_examples),"Y Shape", np.shape(labels_train))
print("Weights Shape",np.shape(w),"Baisies shape",np.shape(b))
optimizer(w,b,standardized_train_examples,labels_train,.1,10)

In [ ]:
def predictor(X,w,b):
    # b = scalar
    # w = (____, 1)
    # X = (____, examples)
    w_transposed = w.T
#     print(np.shape(w_transposed),np.shape(X))
#     new_w = w_transposed.reshape(X.shape[0], 1)
#     print(new_w.shape)
#     print(np.reshape(w_transposed, X[0]))
    prediction = (np.dot(w_transposed,X)) + b
    prediction = sigmoid(prediction)
    prediction *= 2.0
    prediction = np.floor(prediction)
#     print(np.shape(prediction))
    return prediction
    


In [ ]:
predict_examples = (standardized_train_examples)
# for item in predict_examples:
#     print(np.shape(item))
predictor(predict_examples,w,b)

In [123]:
def logisticRegression(train_X,train_Y,test_X,test_Y,learning_rate,epoch):
    # train_x = (image_width * image_width * 3, examples)
    # train_y = (1, examples)
    # test_x = (____, examples)
    # test_y = (1, examples)

    print(train_X.shape)
    
    num_of_examples =  np.shape(train_Y)[0]
#     train_Y = train_Y.reshape(num_of_examples,1)
#     train_Y = train_Y.T 
    w,b = initialize_zeros(side_length*side_length*3)
    print("w logistic regression",w)
#     print("shapes before optimizer",np.shape(train_X),np.shape(train_Y))
    print("shapes before optimizer IIIIIII")
    print("X Shape", np.shape(train_X),"Y Shape", np.shape(train_Y))
    print("Weights Shape",np.shape(w),"Baisies shape",np.shape(b))
    _, params,_ = optimizer(w, b, train_X, train_Y, learning_rate, epoch)
    w,b = params["w"], params["b"]
    prediction_train = predictor(train_X,w,b)
    accuracy_train = np.mean(np.abs(prediction_train - train_Y))
    prediction_test = predictor(test_X,w,b)
    accuracy_test = np.mean(np.abs(prediction_test - test_Y))
    print("Train Accuracy:", accuracy_train)
    print("Actual Accuracy:", accuracy_test)
        

In [130]:
train_X = standardized_train_examples
train_Y = labels_train
test_X = standardized_test_examples
test_Y = labels_test
learning_rate = .1
epoch = 10
logisticRegression(train_X,train_Y,test_X,test_Y,learning_rate,epoch)

(1267500, 21)
w logistic regression [[ 0.]
 [ 0.]
 [ 0.]
 ..., 
 [ 0.]
 [ 0.]
 [ 0.]]
shapes before optimizer IIIIIII
X Shape (1267500, 21) Y Shape (21,)
Weights Shape (1267500, 1) Baisies shape ()
sigmoid input [[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.]]
sigmoid output [[ 0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
   0.5  0.5  0.5  0.5  0.5  0.5  0.5]]
propgate: a [[ 0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5  0.5
   0.5  0.5  0.5  0.5  0.5  0.5  0.5]]


TypeError: 'str' object cannot be interpreted as an integer

Model flow:
(1) logistic Regression
    (2) optimizer
        (3) proprogate 
    (4) predict